In [ ]:
from proteomicRuler.ruler import Ruler, add_mw
import pandas as pd

Preparing the dataframe for the operation

In [ ]:
accession_id_col = "Protein IDs"
# used as unique index and to directly fetch mw data from UniProt

mw_col = "Mass"
# molecular weight column name

ploidy = 2
# ploidy number

total_cellular_protein_concentration = 200
# cellular protein concentration used for calculation of total volume

filename = r"C:\Users\Toan Phung\Downloads\proteinGroups.txt"
df = pd.read_csv(filename, sep="\t")

# selecting intensity columns
intensity_columns = df.columns[57:57+16]

# filter out contaminants and low confident data
df = df[(df["Only identified by site"]!="+")&(df["Only identified by site"]!="Reverse")&(df["Only identified by site"]!="Potential contaminant")]

# calculate tmt cumulative intensity
df["cumulative_intensity"] = df[intensity_columns].sum(axis=1)

# recalculate corrected reporter intensity
for i, r in df.iterrows():
    if r["cumulative_intensity"] != 0:
        for c in intensity_columns:
            df.at[i, c] = r[c]/r["cumulative_intensity"]*r["Intensity"]

Add molecular weight data from UniProt to the original dataframe and remove data that does not have molecular weight

In [ ]:
df = add_mw(df, accession_id_col)
df = df[pd.notnull(df[mw_col])]
df[mw_col] = df[mw_col].str.replace(",", "")
df[mw_col] = df[mw_col].astype(float)

Perform copy number analysis and generate output dataframe and associated rank plots

In [ ]:
ruler = Ruler(df, intensity_columns, mw_col, accession_id_col, ploidy, total_cellular_protein_concentration)
ruler.df.to_csv(filename+"output.tsv", sep="\t", index=False)